In [1]:
import numpy as np

import os
from os.path import  join

In [2]:
#clean_data_path = "D://test_neurolab//train//clean"
clean_data_path = "C://input//neurolab//train//train//clean"

In [3]:
st = 0 
clean_data = []
for folder in os.listdir(clean_data_path):
    for file in os.listdir(join(clean_data_path, folder)):
        data = np.load(join(clean_data_path, folder, file))
        clean_data.append(data)
        #clead_data_sizes.append(data.size)
        #clead_data_shape.append(data.shape)

In [4]:
#noisy_data_path = "D://test_neurolab//train//noisy"
noisy_data_path = "C://input//neurolab//train//train//noisy"

In [5]:
noisy_data = []
for folder in os.listdir(noisy_data_path):
    for file in os.listdir(join(noisy_data_path, folder)):
        data= np.load(join(noisy_data_path, folder, file))
        noisy_data.append(data)

In [7]:
ex_data = clean_data[1]

In [6]:
def get_rect_windows(mel, debug = False):
    if (debug): print("mel size", mel.shape)
    windows = []
    for i in range(mel.shape[0]//80 + 1):
        if (80*(i+1)-1 < mel.shape[0]):
            if (debug): print("current size", 80*i, 80*(i+1)-1)
                
            ixgrid = np.ix_(range(80*i, 80*(i+1)), range(80))
            sample = mel[ixgrid]
            sample = sample.flatten()
            windows.append(sample)
            
            if (debug): print(sample.shape)
        elif(80 - (80*(i+1)-1 - mel.shape[0]) > 30): 
            
            if (debug): print("current size", 80*i, 80*(i+1)-1)
            ixgrid = np.ix_(range(80*i, mel.shape[0]), range(80))
            if (debug): print(mel[ixgrid].shape)                
            pad_matrix = np.pad(mel[ixgrid], ((0, 80 - mel[ixgrid].shape[0]), (0, 0)), 'reflect')
            
            pad_matrix = pad_matrix.flatten()  
            if (debug): print(pad_matrix.shape) 
            windows.append(pad_matrix)
        else:
            if (debug): print("not enough real data left", 80 - (80*(i+1)-1 - mel.shape[0]))
    return windows
a = get_rect_windows(ex_data, debug = True)

NameError: name 'ex_data' is not defined

In [7]:
clean_data_wind = []
for mel in clean_data:
    new_windows = get_rect_windows(mel)
    clean_data_wind.extend(new_windows)

In [8]:
noisy_data_wind = []
for mel in noisy_data:
    new_windows = get_rect_windows(mel)
    noisy_data_wind.extend(new_windows)

In [27]:
len(clean_data_wind),len(noisy_data_wind)

(115403, 115403)

In [33]:
a = [1,2] 
b = [2,3]
c = a.extend(b)
a,c

([1, 2, 2, 3], None)

In [9]:
y_train_clean = [0 for _ in range(len(clean_data_wind))]
y_train_noisy = [1 for _ in range(len(clean_data_wind))]
y = y_train_clean + y_train_noisy

In [39]:
len(y_train)

230806

In [10]:
X = clean_data_wind + noisy_data_wind

In [43]:
len(X)

230806

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import pickle

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV

In [36]:
len(X),len(y)

(230806, 230806)

In [ ]:
logreg = LogisticRegression(class_weight='balanced', verbose = 15)
param = {'C':[0.001,0.003,0.005,0.01,0.03,0.05,0.1,0.3,0.5,1,2,3,3,4,5,10,20]}
clf = GridSearchCV(logreg,param,scoring='roc_auc',refit=True,cv=10)
clf.fit(X,y)

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

In [13]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression(verbose = 5)
logreg.fit(X_train, y_train)

[LibLinear]

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=5, warm_start=False)

In [ ]:
filename = 'logreg_gscv.sav'
pickle.dump(clf, open(filename, 'wb'))

In [16]:
filename = 'logreg_init.sav'
pickle.dump(logreg, open(filename, 'wb'))

In [17]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.85


In [19]:

confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[28731  5808]
 [ 4720 29983]]


In [21]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.86      0.83      0.85     34539
          1       0.84      0.86      0.85     34703

avg / total       0.85      0.85      0.85     69242



In [23]:
ex_data = clean_data[1]
ex_data.shape

(658, 80)

In [26]:
logreg.predict(noisy_data_wind[0].reshape(1, -1))

array([1])

In [31]:
def predict_by_mel(mel, model,debug = False):
    if (debug): print("mel size", mel.shape)
    predictions = []
    for i in range(mel.shape[0]//80 + 1):
        if (80*(i+1)-1 < mel.shape[0]):
            if (debug): print("current size", 80*i, 80*(i+1)-1)
                
            ixgrid = np.ix_(range(80*i, 80*(i+1)), range(80))
            sample = mel[ixgrid]
            sample = sample.flatten()
            
            current_prediction = model.predict(sample.reshape(1, -1))
            
            predictions.append(current_prediction)
            
            #if (debug): print(sample.shape)
            if (debug): print(current_prediction)
        elif(80 - (80*(i+1)-1 - mel.shape[0]) > 30): 
            
            if (debug): print("current size", 80*i, 80*(i+1)-1)
            ixgrid = np.ix_(range(80*i, mel.shape[0]), range(80))
            if (debug): print(mel[ixgrid].shape)                
            pad_matrix = np.pad(mel[ixgrid], ((0, 80 - mel[ixgrid].shape[0]), (0, 0)), 'reflect')
            pad_matrix = pad_matrix.flatten()  
            if (debug): print(pad_matrix.shape) 
            current_prediction = model.predict(pad_matrix.reshape(1, -1))   
            windows.append(current_prediction)
        else:
            if (debug): print("not enough real data left", 80 - (80*(i+1)-1 - mel.shape[0]))
    av_predictions = np.mean(predictions) 
    if (debug) : print(predictions)   
    if(av_predictions > 0.5):
        final_prediction = 1
    else:
        final_prediction = 0
         
    return final_prediction
predict_by_mel(noisy_data[0], logreg, debug = True)

mel size (966, 80)
current size 0 79
[1]
current size 80 159
[1]
current size 160 239
[1]
current size 240 319
[1]
current size 320 399
[1]
current size 400 479
[1]
current size 480 559
[1]
current size 560 639
[1]
current size 640 719
[1]
current size 720 799
[1]
current size 800 879
[1]
current size 880 959
[1]
not enough real data left 7
[array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1])]


1

# VALIDATION

In [ ]:
clean_data_path_val = "C://input//neurolab//val//clean"
clean_data_val = []
for folder in os.listdir(clean_data_path_val):
    for file in os.listdir(join(clean_data_path_val, folder)):
        data = np.load(join(clean_data_path_val, folder, file))
        clean_data_val.append(data)
        
noisy_data_path_val = "C://input//neurolab//val//noisy"
noisy_data_val = []
for folder in os.listdir(noisy_data_path_val):
    for file in os.listdir(join(noisy_data_path_val, folder)):
        data= np.load(join(noisy_data_path_val, folder, file))
        noisy_data_val.append(data)

clean_data_wind_val = []
for mel in clean_data_val:
    new_windows = get_rect_windows(mel)
    clean_data_wind_val.extend(new_windows)
    
noisy_data_wind_val = []
for mel in noisy_data_val:
    new_windows = get_rect_windows(mel)
    noisy_data_wind_val.extend(new_windows)
    
y_train_clean_val = [0 for _ in range(len(clean_data_wind_val))]
y_train_noisy_val = [1 for _ in range(len(clean_data_wind_val))]
y = y_train_clean_val + y_train_noisy_val

X_val = clean_data_wind_val + noisy_data_wind_val